In [2]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests
import asyncio
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type


def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [3]:
def col_to_index(col):
    index = 0
    for c in col:
        index = index * 26 + (ord(c) - ord('A') + 1)
    return index - 1

In [4]:
def convert_range_to_indices(sheet_id, cell_range):
    """
    주어진 셀 범위를 rowIndex와 columnIndex로 변환합니다.
    
    :param sheet_id: 변환할 시트의 ID
    :param cell_range: 변환할 셀 범위 (예: "A2:B")
    :return: 변환된 범위 딕셔너리
    """
    import re
    
    # A1 표기법을 정규 표현식으로 분리
    match = re.match(r"([A-Z]+)([0-9]+):([A-Z]+)([0-9]+)", cell_range)
    if not match:
        raise ValueError("셀 범위 형식이 올바르지 않습니다. 예: 'A2:B10'")
    
    start_col, start_row, end_col, end_row = match.groups()
    
    start_row_index = int(start_row) - 1
    end_row_index = int(end_row)
    start_col_index = col_to_index(start_col)
    end_col_index = col_to_index(end_col) + 1
    
    return {
        'sheetId': sheet_id,
        'startRowIndex': start_row_index,
        'endRowIndex':end_row_index,
        'startColumnIndex': start_col_index,
        'endColumnIndex': end_col_index
    }

In [5]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_to_function(formula, cell_range, sheet_order_id):
    range_name = f"{cell_range}"  # 지정된 시트의 특정 셀 범위    
    # spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    range_value = convert_range_to_indices(sheet_order_id, range_name);

    request = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {
                            "formulaValue": formula
                        }}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": range_value,
        }
    }
    return request

In [6]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def batchWriteRequest(service, spreadsheet_id, all_requests):
    body = {
        'requests': all_requests
    }

    # batchUpdate 호출
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=body
    ).execute()
    
    return response


In [7]:
def fetch_all_sheet_ids(service, spreadsheet_id):
    # 스프레드시트 정보를 가져옵니다.
    spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id, fields='sheets(properties(sheetId,title))').execute()
    
    # 시트 이름과 시트 ID를 딕셔너리로 매핑합니다.
    sheets = spreadsheet.get('sheets', [])
    sheet_dict = {sheet['properties']['title']: sheet['properties']['sheetId'] for sheet in sheets}

    return sheet_dict

In [9]:
IMP_SHEET_NAME_OBJECT_19_281 = {'IMP_1': 1709209987, '셀러코드': 1687831749, '셀러코드변환': 126595561, '은행여기다': 645233419, '잔액': 556778718, '20번정산': 540612375, '19_발주서': 1440280799, '20_발주서': 634245639, '21_발주서': 447955615, '22_발주서': 64878194, '23_발주서': 1787540201, '24_발주서': 1595742130, '25_발주서': 1168070586, '26_발주서': 1165886914, '27_발주서': 874972075, '28_발주서': 1329859700, '29_발주서': 862557386, '30_발주서': 1529111714, '31_발주서': 2022617491, '32_발주서': 1105470909, '33_발주서': 916802662, '34_발주서': 1238434307, '35_발주서': 629282097, '36_발주서': 1117341648, '37_발주서': 870669630, '38_발주서': 1698528401, '39_발주서': 1765463164, '40_발주서': 1391971359, '41_발주서': 386908984, '42_발주서': 2047572301, '43_발주서': 787273921, '44_발주서': 1716246051, '45_발주서': 1169904579, '46_발주서': 1499177987, '47_발주서': 396095253, '48_발주서': 290067466, '49_발주서': 857109948, '50_발주서': 421863820, '51_발주서': 407930910, '52_발주서': 395630467, '53_발주서': 1419579988, '54_발주서': 981678381, '55_발주서': 1894347477, '56_발주서': 1171212966, '57_발주서': 175912695, '58_발주서': 248236688, '59_발주서': 1235281832, '60_발주서': 1383038764, '61_발주서': 1250878924, '62_발주서': 2140073073, '64_발주서': 514692251, '65_발주서': 807550413, '66_발주서': 540703447, '67_발주서': 1007168887, '68_발주서': 1838375110, '69_발주서': 504321830, '70_발주서': 62646797, '71_발주서': 664797007, '72_발주서': 579101005, '73_발주서': 1831155601, '74_발주서': 1794208343, '75_발주서': 1502811150, '76_발주서': 642529460, '77_발주서': 644458190, '78_발주서': 1135330705, '79_발주서': 499905734, '80_발주서': 3274598, '81_발주서': 1396388806, '82_발주서': 1343962798, '83_발주서': 957391009, '84_발주서': 155871732, '85_발주서': 663011650, '86_발주서': 965873049, '87_발주서': 125078810, '88_발주서': 2133096807, '89_발주서': 1183622739, '90_발주서': 1252573650, '91_발주서': 1385152117, '92_발주서': 1595352063, '93_발주서': 1196959023, '94_발주서': 440125658, '95_발주서': 1508703343, '96_발주서': 1916434092, '97_발주서': 1028233600, '98_발주서': 1203818493, '99_발주서': 792860074, '100_발주서': 1826789563, '101_발주서': 69415166, '102_발주서': 1433959745, '103_발주서': 1265051213, '104_발주서': 1481402388, '105_발주서': 790988358, '106_발주서': 1391952505, '107_발주서': 108586903, '108_발주서': 693447041, '109_발주서': 833357576, '110_발주서': 1531918441, '111_발주서': 783937028, '112_발주서': 2070371640, '113_발주서': 1514983711, '114_발주서': 1116763162, '115_발주서': 1662387097, '116_발주서': 632777486, '117_발주서': 1301284685, '118_발주서': 1169144174, '119_발주서': 1656350403, '120_발주서': 631196368, '121_발주서': 883570887, '122_발주서': 1951323945, '123_발주서': 411632150, '124_발주서': 1719331338, '125_발주서': 1316774775, '126_발주서': 276165321, '127_발주서': 1486383677, '128_발주서': 1933516098, '129_발주서': 1896701751, '130_발주서': 26059119, '131_발주서': 1408679578, '132_발주서': 910943055, '133_발주서': 983049740, '134_발주서': 1320954586, '135_발주서': 564067768, '136_발주서': 28777597, '137_발주서': 2130268810, '138_발주서': 1896914313, '139_발주서': 1387520748, '140_발주서': 182106627, '141_발주서': 1669018364, '142_발주서': 1814482987, '143_발주서': 1244767695, '144_발주서': 1638633404, '145_발주서': 1811474991, '146_발주서': 1701264181, '147_발주서': 319610751, '148_발주서': 2036490360, '149_발주서': 552609178, '150_발주서': 1191567224, '151_발주서': 906067570, '152_발주서': 444654631, '153_발주서': 2046172322, '154_발주서': 366195091, '155_발주서': 514902941, '156_발주서': 1940426316, '157_발주서': 24801512, '158_발주서': 453291881, '159_발주서': 1786698510, '160_발주서': 1608013783, '161_발주서': 1794149933, '162_발주서': 1595362854, '163_발주서': 1125337854, '164_발주서': 1474948932, '165_발주서': 621262507, '166_발주서': 1869920807, '167_발주서': 1198945274, '168_발주서': 442354532, '169_발주서': 994810828, '170_발주서': 158409552, '171_발주서': 1029810482, '172_발주서': 1698650592, '173_발주서': 1568661861, '174_발주서': 99349371, '175_발주서': 507546765, '176_발주서': 1402859407, '177_발주서': 15814298, '178_발주서': 167418037, '179_발주서': 1879838961, '180_발주서': 7572723, '181_발주서': 2070990341, '182_발주서': 1779012668, '183_발주서': 1456561385, '184_발주서': 1074096377, '185_발주서': 1255463607, '186_발주서': 88160543, '187_발주서': 16389022, '188_발주서': 1940120811, '189_발주서': 1878854091, '190_발주서': 1104381579, '191_발주서': 2084356380, '192_발주서': 1528149122, '193_발주서': 1928510828, '194_발주서': 486689057, '195_발주서': 1100733534, '196_발주서': 301466999, '197_발주서': 47650216, '198_발주서': 324766328, '199_발주서': 857257789, '200_발주서': 775830565, '201_발주서': 857345941, '202_발주서': 1649177584, '203_발주서': 131380085, '204_발주서': 2035040824, '205_발주서': 1316338785, '206_발주서': 2122589586, '207_발주서': 117224044, '208_발주서': 771316302, '209_발주서': 468964570, '210_발주서': 1669626794, '211_발주서': 1147108982, '212_발주서': 1732761869, '213_발주서': 1099214158, '214_발주서': 1680804784, '215_발주서': 551139338, '216_발주서': 462402494, '217_발주서': 344344671, '218_발주서': 858517879, '219_발주서': 1593530609, '220_발주서': 367704595, '221_발주서': 683925561, '222_발주서': 938929308, '223_발주서': 1643072462, '224_발주서': 2040534877, '225_발주서': 696764743, '226_발주서': 540722520, '227_발주서': 1806626004, '228_발주서': 1718162909, '229_발주서': 1133398499, '230_발주서': 2110535200, '231_발주서': 1522771991, '232_발주서': 1038783182, '233_발주서': 365131131, '234_발주서': 1266156170, '235_발주서': 1878189410, '236_발주서': 593540801, '237_발주서': 523998777, '238_발주서': 1690310620, '239_발주서': 1133568422, '240_발주서': 1130824293, '241_발주서': 1930815496, '242_발주서': 1906412331, '243_발주서': 1338075482, '244_발주서': 162293471, '245_발주서': 920661760, '246_발주서': 1166028658, '247_발주서': 134655683, '248_발주서': 18032810, '249_발주서': 655415764, '250_발주서': 773718264, '251_발주서': 851891952, '252_발주서': 1200920087, '253_발주서': 789318913, '254_발주서': 691213226, '255_발주서': 1222773181, '256_발주서': 1788244242, '257_발주서': 1749739719, '258_발주서': 477831842, '259_발주서': 1875944003, '260_발주서': 419749340, '261_발주서': 103204490, '262_발주서': 1032231043, '263_발주서': 106386279, '264_발주서': 981864553, '265_발주서': 868400508, '266_발주서': 480526854, '267_발주서': 598415891, '268_발주서': 1358386448, '269_발주서': 1896508772, '270_발주서': 1420777506, '271_발주서': 1514063820, '272_발주서': 1134442638, '273_발주서': 640921596, '274_발주서': 2048392347, '275_발주서': 661516976, '276_발주서': 1822879465, '277_발주서': 374772278, '278_발주서': 648647481, '279_발주서': 864617209, '280_발주서': 1867593372, '281+_발주서': 266239839, '502+_발주서': 1778828707}

IMP_SHEET_NAME_OBJECT_19_282_501 = {'IMP_1': 1709209987, '함수': 0, '셀러코드': 1687831749, '셀러코드변환': 126595561, '은행여기다': 645233419, '잔액': 556778718, '281_발주서': 1143954184, '282_발주서': 1840705047, '283_발주서': 1846062935, '284_발주서': 591737751, '285_발주서': 1337880923, '286_발주서': 67906459, '287_발주서': 554137181, '288_발주서': 1765799504, '289_발주서': 1700562611, '290_발주서': 1933361426, '291_발주서': 428063063, '292_발주서': 881456432, '293_발주서': 1004264764, '294_발주서': 1922782233, '295_발주서': 1353586768, '296_발주서': 2018179381, '297_발주서': 1084347371, '298_발주서': 1270203845, '299_발주서': 1725626441, '300_발주서': 1253900206, '301_발주서': 2081645556, '302_발주서': 920468285, '303_발주서': 1554294498, '304_발주서': 742370007, '305_발주서': 763211053, '306_발주서': 987925998, '307_발주서': 1070637413, '308_발주서': 663165543, '309_발주서': 394261056, '310_발주서': 1616828602, '311_발주서': 26608714, '312_발주서': 752356812, '313_발주서': 88012693, '314_발주서': 1548111641, '315_발주서': 1441771347, '316_발주서': 1068385171, '317_발주서': 899680896, '318_발주서': 488756823, '319_발주서': 1246666744, '320_발주서': 340185688, '321_발주서': 1260853487, '322_발주서': 371453681, '323_발주서': 1202474755, '324_발주서': 725182999, '325_발주서': 219738111, '326_발주서': 842960538, '327_발주서': 307692302, '328_발주서': 1885949211, '329_발주서': 345081789, '330_발주서': 1788908953, '331_발주서': 88595591, '332_발주서': 1195516670, '333_발주서': 1310059102, '334_발주서': 1117405498, '335_발주서': 1868245442, '336_발주서': 963427176, '337_발주서': 1399636795, '338_발주서': 192330261, '339_발주서': 742956635, '340_발주서': 1917057985, '341_발주서': 1157815935, '342_발주서': 1940812594, '343_발주서': 268756033, '344_발주서': 293991094, '345_발주서': 1464635047, '346_발주서': 1238249915, '347_발주서': 365696488, '348_발주서': 1465652061, '349_발주서': 1819703786, '350_발주서': 1711258439, '351_발주서': 347527351, '352_발주서': 326844802, '353_발주서': 390076697, '354_발주서': 861755394, '355_발주서': 401980413, '356_발주서': 2128574218, '357_발주서': 669506842, '358_발주서': 1192268674, '359_발주서': 1303573217, '360_발주서': 38677145, '361_발주서': 948173994, '362_발주서': 1530946202, '363_발주서': 49914428, '364_발주서': 660163779, '365_발주서': 443541756, '366_발주서': 1752573055, '367_발주서': 642969820, '368_발주서': 1056295301, '369_발주서': 1608896385, '370_발주서': 546808309, '371_발주서': 1883095957, '372_발주서': 37661729, '373_발주서': 331061238, '374_발주서': 208088198, '375_발주서': 382776475, '376_발주서': 1079639645, '377_발주서': 888861674, '378_발주서': 630141948, '379_발주서': 674887655, '380_발주서': 1944211375, '381_발주서': 1610622189, '382_발주서': 1596640997, '383_발주서': 612651407, '384_발주서': 921078229, '385_발주서': 1723005745, '386_발주서': 860355043, '387_발주서': 1976740580, '388_발주서': 1173341878, '389_발주서': 439764404, '390_발주서': 1857499147, '391_발주서': 433672685, '392_발주서': 449739230, '393_발주서': 399000291, '394_발주서': 1261616353, '395_발주서': 476322614, '396_발주서': 1787226648, '397_발주서': 691749242, '398_발주서': 1393452581, '399_발주서': 1454702576, '400_발주서': 1925467264, '401_발주서': 636144287, '402_발주서': 1005649537, '403_발주서': 492962746, '404_발주서': 1825590402, '405_발주서': 2106235793, '406_발주서': 1668620589, '407_발주서': 1569882576, '408_발주서': 1757166065, '409_발주서': 1877182134, '410_발주서': 74323186, '411_발주서': 1595990512, '412_발주서': 910745892, '413_발주서': 1515404458, '414_발주서': 1982715801, '415_발주서': 751154686, '416_발주서': 1169820214, '417_발주서': 1770257863, '418_발주서': 902385904, '419_발주서': 1125203648, '420_발주서': 593665972, '421_발주서': 1377981914, '422_발주서': 1408882798, '423_발주서': 1157306951, '424_발주서': 2084458302, '425_발주서': 279513233, '426_발주서': 439027907, '427_발주서': 390123867, '428_발주서': 771023481, '429_발주서': 1135729975, '430_발주서': 1417275205, '431_발주서': 557231003, '432_발주서': 35083610, '433_발주서': 1431958704, '434_발주서': 348591994, '435_발주서': 1757784150, '436_발주서': 907216872, '437_발주서': 44246713, '438_발주서': 124011788, '439_발주서': 1121130671, '440_발주서': 1231383906, '441_발주서': 1210655642, '442_발주서': 1591296968, '443_발주서': 1925724643, '444_발주서': 1063349994, '445_발주서': 1008351364, '446_발주서': 1754102771, '447_발주서': 2024299374, '448_발주서': 1578472136, '449_발주서': 1359281384, '450_발주서': 1864132130, '451_발주서': 1748308116, '452_발주서': 455819775, '453_발주서': 679969246, '454_발주서': 652669159, '455_발주서': 1014916137, '456_발주서': 813403067, '457_발주서': 691457311, '458_발주서': 1733818644, '459_발주서': 1596337180, '460_발주서': 1823054931, '461_발주서': 1321228947, '462_발주서': 422415930, '463_발주서': 811000917, '464_발주서': 1881485351, '465_발주서': 245095597, '466_발주서': 39844591, '467_발주서': 466098186, '468_발주서': 481685501, '469_발주서': 1113050160, '470_발주서': 1603148038, '471_발주서': 2138025833, '472_발주서': 1683895598, '473_발주서': 2146801017, '474_발주서': 722691400, '475_발주서': 560748820, '476_발주서': 1988325197, '477_발주서': 1879671545, '478_발주서': 327933425, '479_발주서': 801611412, '480_발주서': 1764000310, '481_발주서': 2074565437, '482_발주서': 223410876, '483_발주서': 1072170942, '484_발주서': 1802918262, '485_발주서': 1376222217, '486_발주서': 551717060, '487_발주서': 1273321746, '488_발주서': 774305924, '489_발주서': 157620174, '490_발주서': 450817660, '491_발주서': 1229163540, '492_발주서': 350742432, '493_발주서': 1832544594, '494_발주서': 691112118, '495_발주서': 653657746, '496_발주서': 2079336401, '497_발주서': 1107387359, '498_발주서': 476087165, '499_발주서': 1676077888, '500_발주서': 1111623301, '501_발주서': 41159966, '502_발주서': 1540009866}

IMP_SHEET_NAME_OBJECT_19_502 = {'IMP_1': 1709209987, '함수': 0, '셀러코드': 1687831749, '셀러코드변환': 126595561, '은행여기다': 645233419, '잔액': 556778718, '버전2_발주서복붙': 375108508, '버전2_발주서': 2108556618, '502_발주서': 240651404, '503_발주서': 348570229, '504_발주서': 635545605, '505_발주서': 2025028886, '506_발주서': 759875285, '507_발주서': 53300007, '508_발주서': 266387640, '509_발주서': 1865644393, '510_발주서': 869141613, '511_발주서': 1150924845, '512_발주서': 1302626005, '513_발주서': 562004668, '514_발주서': 1945492398, '515_발주서': 723251222, '516_발주서': 1718583944, '517_발주서': 22390408, '518_발주서': 367291467, '519_발주서': 632744169, '520_발주서': 565669656, '521_발주서': 396673532, '522_발주서': 856464795, '523_발주서': 1868726116, '524_발주서': 1957495804, '525_발주서': 1565944431, '526_발주서': 75657738, '527_발주서': 247248200, '528_발주서': 2043751782, '529_발주서': 2006740712, '530_발주서': 382866424, '531_발주서': 1935524427, '532_발주서': 1489865751, '533_발주서': 116916937, '534_발주서': 1396670581, '535_발주서': 817599702, '536_발주서': 1991722092, '537_발주서': 953769526, '538_발주서': 542344693, '539_발주서': 1749909952, '540_발주서': 1755449154, '541_발주서': 2096552811, '542_발주서': 905483296, '543_발주서': 343359712, '544_발주서': 1085060633, '545_발주서': 1354773362, '546_발주서': 2075797790, '547_발주서': 1074477384, '548_발주서': 493010647, '549_발주서': 1879021808, '550_발주서': 2066914864, '551_발주서': 355335285, '552_발주서': 1738258134, '553_발주서': 964238109, '554_발주서': 2052901135, '555_발주서': 1287988243, '556_발주서': 653951066, '557_발주서': 2141993886, '558_발주서': 1259201406, '559_발주서': 1959476254, '560_발주서': 217089968, '561_발주서': 108196769, '562_발주서': 663312652, '563_발주서': 95722672, '564_발주서': 423165792, '565_발주서': 1734649834, '566_발주서': 1471508995, '567_발주서': 1857825822, '568_발주서': 1890692718, '569_발주서': 431274414, '570_발주서': 1023290435, '571_발주서': 33633801, '572_발주서': 673097356, '573_발주서': 1178392338, '574_발주서': 2133284450, '575_발주서': 2013390269, '576_발주서': 524786076, '577_발주서': 1642926781, '578_발주서': 1706871399, '579_발주서': 800881613, '580_발주서': 96956878, '581_발주서': 2067575527, '582_발주서': 912839558, '583_발주서': 1205274710, '584_발주서': 430684275, '585_발주서': 1463080362, '586_발주서': 1701000418, '587_발주서': 1518340481, '588_발주서': 2097183862, '589_발주서': 1815714890, '590_발주서': 499762148}

In [13]:
data_list = [
    # ['1', '1XYx2lJZAYpH2pdFiMPA-3QV9rLNtQ4S7F27d5msEEno', '최재혁'], ['3', '1G1qctmwr3pMKrhMHv1hMbvXrmmCI1TsJ5OLqKKg7U8c', '백민기'], 
    ['20', '1D86pGcpOsFWp6IWb_KCGnNJerYcIiFNYzptaR2m21WI', '권오상(오에스인'], 
    # ['21', '1U_HI5Th-1Yh9RBWiOGljs-NhXazxPl1HkbPQ9IZrKzA', '왕선균(오늘곳간)'], 
    ['42', '1ziAp-yyjSgHesi8OKrzl9YxQYufx8Q9rUhUFaT5pWzI', '42'], ['47', '1WgI0QIOdj_NS7JdtIgDdYr-EAI5mTVnjQV3iVm5PP40', '47'], ['48', '1JScmTyIC8YKuRE867hRjiP2_eh4DAVmlJMw-zdhKw2s', '48랑랑이네'], ['50', '1MOpKL9N6ZhdqR6OBEq7AL2YekDCZeLs1VOJv35VphEw', '장진웅'], ['56', '1XmTgyrxTdK3hcHUm1YwYkoBom51v-y2jZTwC0A1Bn8U', '송용재'], ['57', '1ps2R_HXXHEuoKz7glv6LzlBZVyMNvfYNx23kQOxdnXE', '57'], ['60', '1erbDgg1807ZRZhRsoZJr6uYlGgLpT557dt81JxsYO-w', '60'], ['90', '1ZTgyd5YQjluAL8pLVKVqk-BqrOl6EgST47Gd8OtP_RA', '90'], ['92', '15DEt2XorXfKa-UlJtHsauu2b4dmS58RJ3fx1fS1zwkY', '92'], ['97', '1andpTzI55GWGRXV-JjaAtSIE3m4i7fDCHU2CNSk9TEU', '방성식'], ['98', '1ppOTzTz8_QMET4m2z0i6-aMg136hwfi1VeL2HCACyh4', '강대혁'], ['114', '1mZVlL4QwI2eHByQEpYs0dLYUQX4OcSa4tsdTJPR2VRI', '114'], ['121', '1ZEtoHfB5c0PJe2cE7TnP5Q5j8nLrc8hF7lPpdemH90A', '121'], ['130', '1MZN4QGtYktUqUXd6DFPxfyIustQ2BciBQdZd7-8jiOU', '보연주식회사'], ['138', '1Ync5OXAnvBk847u0dDiNkg-H11Ma3CKWzu036NCvPUg', '138'], ['143', '1LbXFDggoMRGJPUTgYp9I5BRvJZ3Fm20oQVL82ymRUl0', '143'], ['171', '1ZOX9E8BaYZ9IvVYZ1AeIQ7--PW_QYv7uHviHP-bqOyU', '우리과일맛있게맛'], ['178', '1hoPzaAfLCA3sV_WMrgNd_aQlIXEkBty4nwVcDZG1Pq0', '178무지개식탁'], ['180', '1WkE6NCdspuCQVIi2mGBXiA1GaFN05-l_9y9Iymu3Ln0', '180'], ['189', '1O1F3u9rP1w1zPCBwvgBhzOZxZdcHxgh3ZqbsRBBWI14', '안호현(마싯농)'], ['192', '1TSnCCP6nJ4PljZBsoqru23j8d53pfLJf5S0pK6tDAXM', '192_하이푸드'], ['195', '1q_g0-xOyXOm02BLASm7kyasalv6bWGO2RBF-Gbn6gGw', '다오소'], ['208', '1djE04qMRnEYYLwY0y4qEpHnBVWwOBxAimrzLgTuBe6k', '２０８'], ['209', '1GBPzy-GuxujWEGkH606Eb72-D8lo4yEKrZP7DrGzWFk', '209'], ['226', '1-J6V0nLHQlxLA8PbTiTXaGt08VlqGv5fo3kfaH35FJE', '김정재'], ['230', '1OxaNX2XUK5CTsqseH6Dnj_3Bhi1FNn_79YAAH1sOdhE', '230'], ['260', '1xD9hJnw7s8BNMuFGpC7uiO2YJsjNPRYQoObvNsxo4sE', '송현우'], ['276', '1V0EBc39eBX1ekf8DGFvCiS5SuVnTm6TehpOkhof7Jlo', '276_김도환(청년'], ['283', '1ldQveZcPwtieVcN-zlIzhXBxO3xnY7523gNNQpnPrfE', '283'], ['325', '14SbtLIs4ZPfiaJh-kRb-TE4RtOcO-gw4knuy3MolLX8', '이호연(컨프래'], ['328', '1wcx04SVtzkmlPl-PvqihBnRYHXnaNZ6EJykoHjME5hQ', '전혜린'], ['330', '1bwyuRqmCmNMr0-0qv_kglHL2olWFx-92MvbF-L8iRZY', '330'], 
    # ['335', '13bZWoE-oHVuPaPQXRYxIjDDAqmpvnqETYuP5S94Re08', '홀핸드'], 
    ['337', '10JQGUWjXCWbhmL7fq8cJfCmtBBIiDaOrqdjh1U1EI_U', '337_박정윤'], ['339', '1GCUBnyUr25uhB3HozLbiUQ5JngLhr7rfoNNR1cxAo58', '339슈거팜'], ['343', '1pd06-Meiiz1sntlIlMd49T6jS9Y6ZiXJU1gRs_hr_bE', '343'], 
    ['353', '1jdRXB72CFDd35tj3Vmr7Ig2OXJfZX3x6wTLl0imas2M', '프리미엄찌니샵'], ['367', '1PXjuzLoiXYK9hLjmiSpsKMxnGXc09N_ZWxoaXPvMe-M', '367'], ['387', '1rNW5Kwv4M59HwihbR5FxiHq8MsvepJQsa1gRzrkmwME', '387'], ['406', '1fBNPp2YdC_xpb3W_4jGkZWRfV7l6_Vn96Vt44LQy8HE', '신윤식'], ['436', '1Ssu9s18Gug3HLRZNp3Hr_TddM2IOcPhlGE7ZjLRPsGo', '김홍곤(주윤)'], ['438', '1zWRhxaoNt8egvqUGmkRRf6KEFccJTDH-HleiKce9ZIM', '김윤성(오손도손)'], ['442', '1E2i1IJRuPAoDmXum5vjrNo65mmaEjIyBKhENEH7VZEE', '임경섭(프루딧)'], ['443', '1IOCubZrxV-dR1uNCFSgP3juhLIlMIMHYVaJACAMu1tY', '서영희(담다컴퍼'], ['461', '104ZWc3RoB0WjkITYQLyFBLvkZ5UywvIJZHoWIha9iTk', '우리동네남매마켓'], ['464', '1JG7u3zTmaH_WfKri2MvNGfUwGua3WAaCj-fXW6vSW2w', '조재훈(퍼플로드)'], ['468', '1dMLFlUP-oXYHnmY-Qm4hK0QUykH6sZyPdPza35HPWwU', '오정애'], ['470', '1oQvBGq3yKYDN9GxaK95bmVO99aVwDn9KGxJ_ceI2BFo', '최주연'], ['474', '1XRUFTiPVeZqLj9p1bw3t-qL-B-jy4Qteah6F1W216MA', '이우람'], ['475', '1ElTPyHmukcSKHDc4snsCcGfciEvOnngzXMB-3ccqdFo', '팔도장터'], ['485', '1vVGjqAgaOFovhHhrcpGkHmGktJug0f0zKgYIERT2FTA', '485정준희'], ['486', '1Uxgzhqwf-PLUE7OO9_Vzpkrr9i1N8PrVmvSpCjy6Pyo', '어썸파머스(주)'], ['489', '1RshPDxepseX4rzT6NxqCjVCDRu-I7zr8qUBgX3AxNQY', '이종철(수수마켓)'], ['496', '1xAcpRH5z51YF3dezUkBwc78vxWPfJOXquCN_4OkcuDM', '주식회사엠디어스'], ['497', '1bIOa6_1v7Jk3t2wAaPyo2G-hACN2iadRuNuVJbYMUVA', '497'], ['498', '1Bzna425shVhelODCpen2FUdiGu24KtR7jQUPNhaZkDY', '배민우'], ['500', '1asRBkSVowjMXIFWkeYZuqyTQmbpUhZi6ht3y66k4HE0', '함영민(과일파는'], ['503', '1np5z7X2XVUyTu1sOH_J0RREVPDNVK8CuN7CG8Jjf1K4', '주식회사와이앤제'], 
    # ['504', '1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw', '504_팜브릿지'], 
    ['506', '1HgYKDeGrRGxwg-j-yVqeZe6sJY1ip5mDJQ2lj4jLPt0', 'SH프레시푸드'], ['513', '1_eWdxS4Xdykn-GbZTTyOdH8f41GEvwURRcisG-PI9XU', '황소영'], ['514', '1Z0BJDjKsvPOrYXKD93NyRLUu--ecxpNue05n_UPLJjc', '주식회사호신호'], ['516', '1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM', '김성헌'], ['517', '133IbYEp-aXQH-ECojwEEak6LKRgPKraS0yqTSvqNvWM', '김가람(과람아저'], ['523', '1y6sJA3EAAqaSLPgZMWYJjY5t2IZLrZGMBTXWI7sroIg', '김지영(동지코퍼'], ['525', '1zyM4XOmz1hRUwLM-2WXHh5BaJ-TMowTA0abn8cd7fe4', '525'], ['527', '1RZUWJpKa9b3CiBFNGkgUwHV2nJo1BE1gs9Uoe1XZmuM', '정민지'], ['537', '1kCgT5DWV1CcHTLJldd6BgiNU-Ara1zkX1TFKDMXiSms', '바이티존'], ['538', '1c3mQbrzmSfS19tAA7MWtVyP_DoT47KI3MPKhI24TUhE', '김규리,김경률'], ['541', '17sd8bF7sHRXpj8cEfTJ_44zhn8ALvSqJ26hkZlpU5ik', '그록정산금'], ['542', '1sClq5Zx710IIh5GhHzm6SdIgeZGmHVbmb8rNetSytJM', '조성현'], ['543', '1fmpcNUCK3wxx1M5TvCt1Cr00wtAaFwoUwPIFMnir5xw', '543_최혜정'], ['545', '1iC421AvucDQacamyihRfIP5WymejCEOC6Fd8t2vkre0', '최정훈(조니마켓)'], 
    ['546', '1xUh--GSPTPLmMXXyHLiJ6DjyL3J9BG3py3SeSGJHkcw', '김혜연(어랑,과일'], ['547', '17A2-P3z_NA1BEGUXxRPEXHmpzRgKCmeA3hDuUWyQ3Xk', '이철호'], ['552', '1zt-VXJNilmSDYDzEhZqyV12Ern3WVu2-IivHWSJ4H1o', '가순성'], ['553', '1SCC7A90vXM5srUZpubCyc02Px3RsCpOeUjC8_Nl5g88', '김지인'], ['554', '1F1N6d2Co-E7LbNfnuj_xfTcJczBdUcVjQugp8sUi_3o', '싱그린청과'], ['555', '1Pu9P4zFoDdFXK9ER_rZaEj2fa0FX36YEyCZIAV-YkEw', '(주)마니쿡스'], ['559', '1PLYZbwJy_4EqiWoQw7lMAC6RGc9f6-BEN92yZmAWRZI', '박지환(와치맨컴'], ['562', '1jYeZFz3CNgXkBV3z1CsoD8k_e6xH-vZa3M0KboBD8rk', '우리DC마켓']]

excluded_titles = [21, 335, 442, 504, 516, 523, 555]

# IMP_SHEET_ID = "1nWRXCVmrdwaWE0oRqIHlArKwAPwL0pk5nNAjjN5raOo"
IMP_SHEET_ID_19_281 = "1nWRXCVmrdwaWE0oRqIHlArKwAPwL0pk5nNAjjN5raOo"
IMP_SHEET_ID_282_501 = "1bb3MgGgyzSOhCxQ0LMVBVU5u5wjvMNmHTbFOjf1-9hs"
IMP_SHEET_ID_502 = "1PtUIKeUjbEVX8aY8xi2CcSPI1LCKFVrpqgsmXLlCr7w"

# IMP_SHEET_NAME_OBJECT_19_281

# IMP_SHEET_NAME_OBJECT_19_282_501

# IMP_SHEET_NAME_OBJECT_19_502

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)

    # response = fetch_all_sheet_ids(service_sheets, IMP_SHEET_ID_502)
    requests_1 = []
    requests_282 = []
    requests_502 = []

    for data in data_list[2:]:
        spreadsheet_title = data[0]
        spreadsheet_id = data[1]
        sheet_id_by_sheet_title = ""
        key = f'{spreadsheet_title}_발주서'
        if int(spreadsheet_title) in excluded_titles:
            continue

        if 19 > int(spreadsheet_title):
            continue

        if 502 <= int(spreadsheet_title):
            sheet_id_by_sheet_title = IMP_SHEET_NAME_OBJECT_19_502[key]
        elif  282 <= int(spreadsheet_title) < 502 :
            sheet_id_by_sheet_title = IMP_SHEET_NAME_OBJECT_19_282_501[key]
        elif int(spreadsheet_title) < 282:
            sheet_id_by_sheet_title = IMP_SHEET_NAME_OBJECT_19_281[key]


        formula_IMP_IMP_발주서 = f'=IMPORTRANGE("{spreadsheet_id}", "\'발주서이전버전\'!$A$11:$M$999")'
        result_IMP_발주서 = apply_formula_to_function(formula_IMP_IMP_발주서, "A2:A2", sheet_id_by_sheet_title)

        if 502 <= int(spreadsheet_title):
            requests_502.append(result_IMP_발주서)
        elif  282 <= int(spreadsheet_title) < 502 :
            requests_282.append(result_IMP_발주서)
        elif int(spreadsheet_title) < 282:
            requests_1.append(result_IMP_발주서)

    if len(requests_1) > 0:
        await batchWriteRequest(service_sheets, IMP_SHEET_ID_19_281, requests_1)
    if len(requests_282) > 0:
        await batchWriteRequest(service_sheets, IMP_SHEET_ID_282_501, requests_282)
    if len(requests_502) > 0:
        await batchWriteRequest(service_sheets, IMP_SHEET_ID_502, requests_502)
